In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent)) ## ../../covid19datascience* <- set parent level here

In [ ]:
import pandas as pd
from epiweeks import Week
from datetime import timedelta, date
from os.path import join
from unidecode import unidecode

from bulletin import default_input, default_output, root, hoje, data_comeco_pandemia
from bulletin.utils.normalize import normalize_text, normalize_hash
from bulletin.systems.sim import Sim
from bulletin.systems.sivep import Sivep
from bulletin.systems.notifica import Notifica
from bulletin.systems.casos_confirmados import CasosConfirmados

In [ ]:
cc = CasosConfirmados()
cc.load('cc_01_09_2021')

In [ ]:
covid_obitos = cc.df.loc[cc.df['evolucao']==2].copy()
#del cc

In [ ]:
sim = Sim()
sim.load()
# sim.read_all_database_files()
# sim.save(replace=True)

In [ ]:
sim.to_notifica()

In [ ]:
sim_covid_obitos = sim.get_obitos()

In [ ]:
# del sim

In [ ]:
# sivep = Sivep()
# sivep.load()
# sivep.read_all_database_files()
# sivep.save(replace=True)

In [ ]:
# sivep.to_notifica()

In [ ]:
# sivep_covid_obitos = sivep.get_obitos()

In [ ]:
# del sivep

In [ ]:
qualificacao_obitos = pd.read_excel(join(default_input, 'sim', 'qualificacao_obitos.xlsx'))
qualificacao_obitos.shape[0]

In [ ]:
qualificacao_obitos.loc[~(qualificacao_obitos['NUMERODO'].isin(sim_covid_obitos['numero_do'].values))].to_excel(join(default_output, 'correcoes','nao_consta_b342_no_sim.xlsx'), index=False)

In [ ]:
qualificacao_obitos = qualificacao_obitos[['NUMERODO']].rename(columns={'NUMERODO':'numero_do'}).drop_duplicates(['numero_do'])
qualificacao_obitos.shape[0]

In [ ]:
qualificacao_obitos['numero_do'] = qualificacao_obitos['numero_do'].astype('str')

In [ ]:
sim_covid_obitos.loc[sim_covid_obitos['numero_do'].isin(qualificacao_obitos['numero_do'].values)].shape[0]

In [ ]:
qualificacao_obitos_sim = sim_covid_obitos.loc[sim_covid_obitos['numero_do'].isin(qualificacao_obitos['numero_do'].values)]

In [ ]:
obitos_que_nao_estao_no_pickle = qualificacao_obitos_sim.loc[~(
    qualificacao_obitos_sim['hash_mae'].isin(covid_obitos['hash_mae']) |
    qualificacao_obitos_sim['hash_nasc'].isin(covid_obitos['hash_nasc'])
)].copy()

In [ ]:
obitos_para_fechar_na_planilha = cc.df.loc[(
    cc.df['hash_mae'].isin(obitos_que_nao_estao_no_pickle['hash_mae']) &
    cc.df['hash_nasc'].isin(obitos_que_nao_estao_no_pickle['hash_nasc']) &
    cc.df['hash_resid'].isin(obitos_que_nao_estao_no_pickle['hash_resid'])
)].copy()

In [ ]:
obitos_para_fechar_na_planilha.to_excel(join(default_output, 'correcoes','obitos_para_fechar_no_pickle.xlsx'), index=False)

In [ ]:
mudar_municipio_para_fechar_na_planilha = cc.df.loc[(
    (cc.df['hash_mae'].isin(obitos_que_nao_estao_no_pickle['hash_mae']) |
    cc.df['hash_nasc'].isin(obitos_que_nao_estao_no_pickle['hash_nasc'])) &
    (~cc.df['id_notifica'].isin(obitos_para_fechar_na_planilha['id_notifica']))
)].duplicated('hash_mae').copy()

In [ ]:
mudar_municipio_para_fechar_na_planilha.to_excel(join(default_output, 'correcoes','mudar_municipio_para_fechar_na_planilha.xlsx'), index=False)

In [ ]:
mudar_municipio_para_fechar_na_planilha